# Testing the two-level asymptotic form of `Whirl2d`

In [4]:
include(joinpath(Pkg.dir("whirl2d"), "src/Whirl2d.jl"))
import Whirl2d
import Whirl2d:@get
using PyPlot
@get Whirl2d (Systems, Grids, DualPatch, Bodies, TimeMarching, NavierStokes, Process);

### Create a body undergoing a weak oscillatory translation in `x` direction 

In [5]:
nbody = 1
b1 = let Npts = 64,  # total number of points on circle
         rad = 1.0,   # radius of circle
         xcent = [0.0,0.0],  # initial centroid of circle
         ang = 0.0    # initial angle of circle
         Bodies.Circle(Npts,rad,xcent,ang)
end

#b2 = let Npts = 64,  # total number of points on circle
#         rad = 1.0,   # radius of circle
#         xcent = [4.0,0.0],  # initial centroid of circle
#         ang = 0.0    # initial angle of circle
#         Bodies.Circle(Npts,rad,xcent,ang)
#end

Body: number of points = 64, reference point = (0.0,0.0), rotation matrix = [1.0 -0.0; 0.0 1.0]


### Set the streaming motion

In [6]:
sparams = Array{NavierStokes.StreamingParams}(nbody)

ϵ = 0.1  # A/R

sparams[1] = let Ω = 1.0,    # Angular frequency
                 X = 1.0,  # X amplitude
                 Y = 0.0,   # Y amplitude
                 ϕx = 0.0,  # X phase lead
                 ϕy = 0.0  # Y phase lead                
    
    NavierStokes.StreamingParams(Ω, ϵ, X, ϕx, Y, ϕy)
end;
NavierStokes.set_oscil_motion!(b1,sparams[1])

T = 2π/sparams[1].Ω

6.283185307179586

### Create a domain with this body in it

In [7]:
xmin = [-4.0,-4.0]
xmax = [4.0,4.0]
dom = Systems.DualDomain(xmin,xmax)
dom = Systems.add_body(dom,b1)
#dom = Systems.add_body(dom,b2)

Domain: xmin = [-4.0, -4.0], xmax = [4.0, 4.0]
number of bodies = 1
Body: number of points = 64, reference point = (0.0,0.0), rotation matrix = [1.0 -0.0; 0.0 1.0]



#### Add a filling dual grid to this domain
The boundaries of the domain coincide with the boundaries of the dual grid

In [8]:
Δx = 0.08
dom = Systems.add_grid(dom,Δx)
x = Grids.xcell(dom.grid);
y = Grids.ycell(dom.grid);

### Set the physical and time parameters

In [9]:
Re = 40
physparams = let U = [0.0,0.0]
    NavierStokes.set_freestream(U)
end
NavierStokes.set_Re!(physparams,Re)
Δt = min(0.5*Δx,Δx^2*Re)

α = Δt/(Re*Δx^2)
tparams = TimeMarching.TimeParams(Δt,TimeMarching.RK31())

nT = ceil(Int,T/tparams.Δt) # number of time steps in each period

params = (physparams,α,sparams)
tparams

Time step size 0.04

### Compute the grid and body operators
This fills in the LGF and integrating factor tables, computes the interpolation/regularization operators, computes the Schur complement operators and factorizes them, and defines the operators needed for calling the HERK

In [10]:
gops, ops = NavierStokes.set_operators_two_level_body!(dom,params);

LoadError: [91mMethodError: no method matching convert(::Type{AssertionError}, ::String)[0m
Closest candidates are:
  convert([91m::Type{Any}[39m, ::ANY) at essentials.jl:28
  convert(::Type{T}, [91m::T[39m) where T at essentials.jl:29[39m

### Create an initial solution structure

In [11]:
s = NavierStokes.TwoLevelBodySoln(dom)
h = [s]
#TimeMarching.ifherk!(s,tparams,ops)

MethodError: [91mMethodError: no method matching convert(::Type{AssertionError}, ::String)[0m
Closest candidates are:
  convert([91m::Type{Any}[39m, ::ANY) at essentials.jl:28
  convert(::Type{T}, [91m::T[39m) where T at essentials.jl:29[39m

### Advance the solution

In [12]:
@time for i = 1:6*nT
    s = TimeMarching.ifherk!(s,tparams,ops)
    push!(h,s)
end
s

LoadError: [91mMethodError: no method matching string(::Expr)
The applicable method may be too new: running in world age 2754, while current world is 23230.[0m
Closest candidates are:
  string(::Any...) at strings/io.jl:120 (method too new to be called from this world context.)
  string([91m::BigInt[39m) at gmp.jl:568 (method too new to be called from this world context.)
  string([91m::BigFloat[39m) at mpfr.jl:885 (method too new to be called from this world context.)
  ...[39m

### Evaluate the fields of the solution history

In [13]:
@time f = NavierStokes.evaluateFields.(h,dom.grid,gops);
t = map(x -> x.t, f);
ω = map(x -> x.ω, f);
ψ = map(x -> x.ψ, f);
ux = map(x -> x.ux, f);
uy = map(x -> x.uy, f);

LoadError: [91mUndefVarError: gops not defined[39m

#### Average over the last period

In [14]:
ψavg = NavierStokes.average(ψ, length(f)-nT+1:length(f));
ωavg = NavierStokes.average(ω, length(f)-nT+1:length(f));

LoadError: [91mUndefVarError: ψ not defined[39m

### Plot the solution

#### Level 1 solution

In [15]:
contour(x,y,transpose(ω[end][1]),linspace(-5,5,30))
for j = 1:dom.nbody
    xpt = [dom.body[j].x[i][1] for i = 1:dom.body[j].N]
    ypt = [dom.body[j].x[i][2] for i = 1:dom.body[j].N]
    fill(xpt,ypt,color="k",zorder=2)
end
axis("scaled")

LoadError: [91mUndefVarError: ω not defined[39m

In [16]:
contour(x,y,transpose(ψ[end][1]),31)
for j = 1:dom.nbody
    xpt = [dom.body[j].x[i][1] for i = 1:dom.body[j].N]
    ypt = [dom.body[j].x[i][2] for i = 1:dom.body[j].N]
    fill(xpt,ypt,color="k",zorder=2)
end
axis("scaled")

LoadError: [91mUndefVarError: ψ not defined[39m

#### Level 2 solution

In [17]:
contour(x,y,transpose(ω[end][2]),linspace(-5,5,30))
for j = 1:dom.nbody
    xpt = [dom.body[j].x[i][1] for i = 1:dom.body[j].N]
    ypt = [dom.body[j].x[i][2] for i = 1:dom.body[j].N]
    fill(xpt,ypt,color="k",zorder=2)
end
axis("scaled")

LoadError: [91mUndefVarError: ω not defined[39m

In [18]:
contour(x,y,transpose(ψ[end][2]),31)
for j = 1:dom.nbody
    xpt = [dom.body[j].x[i][1] for i = 1:dom.body[j].N]
    ypt = [dom.body[j].x[i][2] for i = 1:dom.body[j].N]
    fill(xpt,ypt,color="k",zorder=2)
end
axis("scaled")

LoadError: [91mUndefVarError: ψ not defined[39m

#### Level 2 mean solution

In [19]:
contour(x,y,ψavg[2],linspace(-0.5,0.5,31))
for j = 1:dom.nbody
    xpt = [dom.body[j].x[i][1] for i = 1:dom.body[j].N]
    ypt = [dom.body[j].x[i][2] for i = 1:dom.body[j].N]
    fill(xpt,ypt,color="k",zorder=2)
end
axis("scaled")

LoadError: [91mUndefVarError: ψavg not defined[39m

In [20]:
contour(x,y,ωavg[2],linspace(-5,5,30))
for j = 1:dom.nbody
    xpt = [dom.body[j].x[i][1] for i = 1:dom.body[j].N]
    ypt = [dom.body[j].x[i][2] for i = 1:dom.body[j].N]
    fill(xpt,ypt,color="k",zorder=2)
end
axis("scaled")

LoadError: [91mUndefVarError: ωavg not defined[39m

### Sample the computed solution

In [21]:
using Interpolations

In [22]:
xsamp = [2.0,1.5]
ysamp = [0.0,1.5]
ψsamp = [Float64[] for x in xsamp]
uxsamp = [Float64[] for x in xsamp]
uysamp = [Float64[] for x in xsamp]
tsamp = Float64[]
for i = 1:length(h)
    ψf = ϵ*ψ[i][1]+ϵ^2*ψ[i][2]
    uxf = ϵ*ux[i][1]+ϵ^2*ux[i][2]
    uyf = ϵ*uy[i][1]+ϵ^2*uy[i][2]
    ψitp = interpolate((Grids.xcell(dom.grid),Grids.ycell(dom.grid)),ψf*Δx,Gridded(Linear()))
    uxitp = interpolate((Grids.xfacex(dom.grid),Grids.yfacex(dom.grid)),uxf,Gridded(Linear()))
    uyitp = interpolate((Grids.xfacey(dom.grid),Grids.yfacey(dom.grid)),uyf,Gridded(Linear()))    
    for j = 1:length(xsamp)
        push!(ψsamp[j],ψitp[xsamp[j],ysamp[j]])
        push!(uxsamp[j],uxitp[xsamp[j],ysamp[j]])
        push!(uysamp[j],uyitp[xsamp[j],ysamp[j]])
    end
end

LoadError: [91mMethodError: no method matching convert(::Type{AssertionError}, ::String)[0m
Closest candidates are:
  convert([91m::Type{Any}[39m, ::ANY) at essentials.jl:28
  convert(::Type{T}, [91m::T[39m) where T at essentials.jl:29[39m

In [23]:
include(joinpath(Pkg.dir("whirl2d"), "src/Streaming.jl"))
import Streaming

In [24]:
p = let Re = Re, ϵ = ϵ   # use the values of these set above
      Streaming.Params(ϵ,Re)
end
g, r = let xmin = -6.0,
      xmax = 6.0,
      N = 301,
      x = [xmin + (j-1)*(xmax-xmin)/(N-1) for i=1:N, j=1:N],
      y = [xmin + (i-1)*(xmax-xmin)/(N-1) for i=1:N, j=1:N],
      Nᵣ = 500,
      rmax = √2*xmax
      Streaming.Grid(x,y), collect(linspace(1,rmax,Nᵣ))
end

LoadError: [91mMethodError: no method matching Streaming.Params(::Float64, ::Int64)[0m
Closest candidates are:
  Streaming.Params(::Any, ::Any, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m) at /Users/jeff/.julia/v0.6/whirl2d/src/Streaming.jl:18
  Streaming.Params(::Float64, [91m::Float64[39m) at /Users/jeff/.julia/v0.6/whirl2d/src/Streaming.jl:32
  Streaming.Params(::Float64, [91m::Float64[39m, [91m::Complex{Float64}[39m, [91m::Complex{Float64}[39m, [91m::Complex{Float64}[39m, [91m::Complex{Float64}[39m, [91m::Complex{Float64}[39m, [91m::Function[39m, [91m::Function[39m, [91m::Function[39m, [91m::Complex{Float64}[39m) at /Users/jeff/.julia/v0.6/whirl2d/src/Streaming.jl:18[39m

In [25]:
s₁ = Streaming.FirstOrder(p,r)
s̄₂ = Streaming.SecondOrderMean(p,r)
s₂ = Streaming.SecondOrder(p,r)

LoadError: [91mUndefVarError: p not defined[39m

In [26]:
tex = 0.0
sfull = Streaming.Evaluate(tex,p,g,s₁,s̄₂,s₂)

LoadError: [91mUndefVarError: p not defined[39m

In [27]:
contour(g.x,g.y,sfull.ω,linspace(-1,1,30))
R = 1.0
zc = R*exp.(im*linspace(0,2π,100))
fill(real(zc),imag(zc),color="k",zorder=2)
axis("scaled")
xlim(0,maximum(g.x))
ylim(0,maximum(g.y))

LoadError: [91mUndefVarError: g not defined[39m

### Sample the history at a few points

In [28]:
gsamp = Streaming.Grid(xsamp,ysamp)

1-dimensional evaluation grid


#### Set the time range over which to evaluate

In [29]:
tr = let t₀ = 0.0,
    t₁ = 20.0*2π,
    Δt = 2π/40
    t₀:Δt:t₁
end

0.0:0.15707963267948966:125.66370614359172

#### Evaluate the exact solution at sample points over selected time range

In [30]:
hex = Streaming.Evaluate(tr,p,gsamp,s₁,s̄₂,s₂);
uxex, uyex = Streaming.cartesian(hex,gsamp);

LoadError: [91mUndefVarError: p not defined[39m

#### Compare the exact solution with the computed solution

In [31]:
isamp = 2  # which sample point is being plotted
plot(hex.t,uxex[isamp],t,uxsamp[isamp])
title("Comparison between exact solution and computed solution at sample point")
legend(["Exact solution","Computed solution"])

LoadError: [91mtype #hex has no field t[39m